In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import psycopg2
from psycopg2.extras import execute_values

In [2]:
# 1️⃣ Завантаження даних з API
start_date = datetime(2025, 1, 1)
end_date = datetime.today()

all_data = []
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")
    url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date={date_str}&json"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for d in data:
            all_data.append({
                "date": d["exchangedate"],
                "currency_code": d["cc"],
                "currency_name": d["txt"],
                "rate": d["rate"]
            })
    else:
        print(f"Помилка {response.status_code} для {date_str}")
    
    current_date += timedelta(days=1)

In [3]:
# 2️⃣ Створення DataFrame
df = pd.DataFrame(all_data)

In [4]:
# 3️⃣ Перевірка та очищення
df = df.drop_duplicates()
df = df.dropna()
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")
df["rate"] = pd.to_numeric(df["rate"], errors="coerce")
df = df[df["rate"] > 0]

In [5]:
# 4️⃣ Підключення до PostgreSQL та створення таблиці
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="991Gfhjkm.",
    port=5432
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS nbu_exchange_rates (
    date DATE,
    currency_code VARCHAR(3),
    currency_name VARCHAR(100),
    rate NUMERIC
)
""")
conn.commit()


In [7]:
# Збереження очищених даних у CSV
df.to_csv("nbu_exchange_rates_clean.csv", index=False, encoding="utf-8-sig")
print("Файл nbu_exchange_rates_clean.csv збережений")

Файл nbu_exchange_rates_clean.csv збережений


In [17]:
# Перетворення у чисті Python типи
df["rate"] = df["rate"].astype(float)
df["date"] = pd.to_datetime(df["date"]).dt.date
df["currency_code"] = df["currency_code"].astype(str)
df["currency_name"] = df["currency_name"].astype(str)

# Перетворимо на список кортежів
values = list(df.itertuples(index=False, name=None))

# Очистимо таблицю перед вставкою (якщо треба)
# cur.execute("TRUNCATE TABLE nbu_exchange_rates;")

# Виконуємо вставку
try:
    execute_values(cur,
        "INSERT INTO nbu_exchange_rates (date, currency_code, currency_name, rate) VALUES %s",
        values
    )
    conn.commit()
    print("✅ Дані вставлені")
except Exception as e:
    conn.rollback()
    print("❌ Помилка вставки:", e)

✅ Дані вставлені
